Import necessary libraries. Feel free to import any more needed dependencies

In [ ]:
import pandas as pd
import sklearn

Read in test and training data

Clean agreed upon features

train perspective models